In [ ]:
import os
import pandas as pd
import numpy as np
import csv
from scipy import spatial
from matplotlib import pyplot as plt

In [ ]:
#Summary

#1st we need dataset for all thre 66K patients based on Covid Table
#Currently we are having demographic info of 117K patients
#Tables we are looking into : demographics, covid_labs, condition, medication, procedure, and result
#Now upload the dataset from the folders and do the following analysis
# Converting the string to numeric
# Plot the distribution (histogram) of each column to see the skewness
# Then finalize the dataset that will be used for further eval

# Our spreadsheet contains variables from multiple research projects
# Various sub project on gi , neuro , pulmonary , cards ,  Endo
# The ultimate goal is to predict which patients will likely die from COVID-19
# The attributes for each patient will be based off of the fields from each row of the spreadsheet
# These fields are pulled from each of the tables demographics, covid_labs, condition, medication, procedure, and result
# For the Data Mining project we will restrict to these tables: demographics, covid_labs and result

# Of the patients we observed in June of 2020, what factors lead to COVID-19 patient death
# Objective: Deceased: 0 vs 1


In [ ]:
##this function is  file loader
def loadCSVfile(file_name, columns_name, separator=None):
    megabyte_div = 1048576
    file_size = os.path.getsize(file_name)/megabyte_div

    # maximum file size in MB to read in one iteration
    file_size_limit = 100

    if file_size > file_size_limit:
        # no of row to read in one iteration
        chunk_size = 10000
        temporary_data_list = []

        # if columns_name is provided, use them. Otherwise read all columns
        if columns_name:
            for chunk in pd.read_csv(file_name, chunksize=chunk_size, sep=',', usecols=columns_name, engine='python' ):
                temporary_data_list.append(chunk)
        else:
            for chunk in pd.read_csv(file_name, chunksize=chunk_size, sep=',', engine='python'):
                temporary_data_list.append(chunk)

        data_frame = pd.concat(temporary_data_list, axis=0)
        del temporary_data_list, chunk
    else:
        if columns_name:
            data_frame = pd.read_csv(file_name, sep=',', usecols=columns_name, engine='python')
        else:
            data_frame = pd.read_csv(file_name, sep=',', engine='python')

    return data_frame

In [ ]:
#extracting the files names from the folder
complete_path = "./result2020"
all_files = glob.glob(complete_path + "/*.csv")

In [ ]:
#uploading the files for the results table with covid patients
complete_result_df = pd.DataFrame()
for file_name in all_files:
    print("FileName {0}".format(file_name))
    seperator = '/t'
    columns_name = ['personid', 'result', 'textvalue', 'numericvalue', 'numericvaluemodifier', 'unitofmeasure', 'servicedate']
    data_frame = loadCSVfile(file_name, columns_name, seperator)
    complete_result_df = pd.concat([complete_result_df, data_frame])

In [ ]:
complete_result_df.info()

In [ ]:
complete_result_df['personid'].value_counts()

In [ ]:
complete_result_df.count()

In [ ]:
complete_result_df.head(n=10)

In [ ]:
file_name = './df_covid_from_result_table_1000.csv'
print('Is the file path available: ', os.path.isfile(file_name))

In [ ]:
seperator = '/t'
columns_name = ['personid', 'result', 'textvalue', 'numericvalue', 'numericvaluemodifier', 'unitofmeasure', 'servicedate']
data_frame = loadCSVfile(file_name, columns_name, seperator)

In [ ]:
data_frame.head(n=10)

In [ ]:
print(data_frame.isna().sum())

In [ ]:
#extracting the files names from the folder
complete_path2 = "./condition"
all_files_2 = glob.glob(complete_path2 + "/*.csv")

In [ ]:
#uploading the files for the results table with covid patients
complete_conditions_df = pd.DataFrame()
for file_name in all_files_2:
    print("FileName {0}".format(file_name))
    seperator = '/t'
    columns_name = ['personid', 'conditioncode', 'condition', 'classification']
    data_frame = loadCSVfile(file_name, columns_name, seperator)
    complete_conditions_df = pd.concat([complete_conditions_df, data_frame])

In [ ]:
complete_conditions_df['personid'].value_counts()

In [ ]:
complete_conditions_df.info()

In [ ]:
complete_conditions_df.head(n=10)

In [ ]:
#extracting the files names from the folder
complete_path3 = "./procedure2020"
all_files_3 = glob.glob(complete_path3 + "/*.csv")
all_files_3

In [ ]:
#uploading the files for the results table with covid patients
complete_procedure_df = pd.DataFrame()
for file_name in all_files_3:
    print("FileName {0}".format(file_name))
    seperator = '/t'
    columns_name = ['personid', 'codetype', 'procedurecode', 'procedure', 'servicestartdate', 'serviceenddate']
    data_frame = loadCSVfile(file_name, columns_name, seperator)
    complete_procedure_df = pd.concat([complete_procedure_df, data_frame])

In [ ]:
complete_procedure_df.head(n=10)

In [ ]:
complete_procedure_df['personid'].value_counts()

In [ ]:
RequiredResultColumn = ('Respiratory rate','Heart rate', 'Systolic blood pressure','Diastolic blood pressure',
                         'Non-invasive mean arterial pressure','Body temperature',
                         'Oxygen saturation in Arterial blood by Pulse oximetry',
                         'Inhaled oxygen concentration','Inhaled oxygen flow rate','Oxygen therapy',
                         'Oxygen/Inspired gas setting [Volume Fraction] Ventilator',
                         'Cholesterol [Mass/volume] in Serum or Plasma',
                         'Cholesterol in HDL [Mass/volume] in Serum or Plasma',
                         'Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation',
                         'Triglyceride [Mass/volume] in Serum or Plasma',
                         'Hemoglobin A1c/Hemoglobin.total in Blood',
                         'Protein [Mass/volume] in Serum or Plasma',
                         'Albumin [Mass/volume] in Serum or Plasma',
                         'Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma',
                         'Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma',
                         'Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma',
                         'Bilirubin.total [Mass/volume] in Serum or Plasma',
                         'Bilirubin.direct [Mass/volume] in Serum or Plasma',
                         'Troponin T.cardiac [Mass/volume] in Serum or Plasma',
                         'Troponin I.cardiac [Mass/volume] in Serum or Plasma',
                         'Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method',
                         'Natriuretic peptide.B prohormone N-Terminal [Mass/volume] in Serum or Plasma',
                         'Natriuretic peptide B [Mass/volume] in Blood',
                         'Urate [Mass/volume] in Urine',
                         'C reactive protein [Mass/volume] in Serum or Plasma',
                         'INR in Platelet poor plasma by Coagulation assay',
                         'Prothrombin time (PT)','aPTT in Platelet poor plasma by Coagulation assay',
                         'Ferritin [Mass/volume] in Serum or Plasma',
                         'Fibrin D-dimer DDU [Mass/volume] in Platelet poor plasma by Immunoassay',
                         'Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma',
                         'Leukocytes [#/volume] in Blood by Automated count',
                         'Neutrophils [#/volume] in Blood by Automated count',
                         'Lymphocytes [#/volume] in Blood by Automated count',
                         'Erythrocytes [#/volume] in Blood by Automated count',
                         'Hematocrit [Volume Fraction] of Blood by Automated count',
                         'Hemoglobin [Mass/volume] in Blood','MCH [Entitic mass] by Automated count',
                         'MCHC [Mass/volume] by Automated count','MCV [Entitic volume] by Automated count',
                         'Platelets [#/volume] in Blood by Automated count',
                         'Creatinine [Mass/volume] in Serum or Plasma',
                         'Urea nitrogen [Mass/volume] in Serum or Plasma',
                         'Sodium [Moles/volume] in Serum or Plasma',
                         'Potassium [Moles/volume] in Serum or Plasma',
                         'Magnesium [Mass/volume] in Serum or Plasma',
                         'Calcium [Mass/volume] in Serum or Plasma',
                         'Calcium.ionized [Moles/volume] in Blood',
                         '25-Hydroxyvitamin D2+25-Hydroxyvitamin D3 [Mass/volume] in Serum or Plasma',
                         'Ammonia [Moles/volume] in Plasma',
                         'Creatine kinase [Enzymatic activity/volume] in Serum or Plasma',
                         'Influenza virus A Ag [Presence] in Unspecified specimen by Immunoassay',
                         'Influenza virus B Ag [Presence] in Unspecified specimen by Immunoassay',
                         'Respiratory syncytial virus RNA [Presence] in Unspecified specimen by NAA with probe detection')

In [ ]:
##extracting the rows based on the selected value of the result column
subset_result_df = complete_result_df.loc[complete_result_df['result'].isin(RequiredResultColumn)]